# Intermixing of cell populations

This notebook explores further the intermixing measure introduced by Martijn Molenaar in [this manuscript](https://www.biorxiv.org/content/10.1101/2022.08.24.505109v1.full). This metric aims to quantify he spatial separation of two known, distinct populations based on their UMAP coordinates. To this end, it analyses the neighborhood of size $n$ (number of closest neighbors) of every individual data point $a$ and calculates its intermixing metric $i$ as the proportions of data points from the other population in this neighborhood. $$i_{a,n}=\frac{n_{other\_population}}{n}=1-\frac{n_{same\_population}}{n}$$ Here, the euclidian distance in UMAP space serves as measure of proximity and the closest $n$ points form the neighborhood of a point. The mean of all point-based intermixing values forms the intermixing metric $I$ of the dataset for a given neighborhood. $$I_n=mean(i_{1,n}, i_{2,n}, i_{3,n}, ..., )$$ Martijn developed this measure as a numeric value for a defined neighborhood with $n=10$. To extend this overview beyond that definition, I examine the measure on the level of different neighborhood sizes and thus in the form of an intermixing curve (similar to ROC or PRC curves). This graphic representations draws the intermixing metric on the y-axis as a function of the neighborhood size (relative to the dataset size: $n_{rel}$) on the x-axis.

To explore the behavior of this metric, it is computed and compared for artificial datasets with different characteristics. 

In [ ]:
from math import ceil
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from tqdm import tqdm



### 1. The intermixing metric

The metric is adapted from Martijns R code with some additions concerning neighborhood size, normalization and parallelization.

In [ ]:
def general_intermixing_metric(locations, labels, ax, label, linestyle = 'solid', color = 'auto', n_datapoints = 100, normalized = True, n_jobs = 6):
    
    dist_matrix = distance_matrix(locations, locations)
    neighborhood_size = len(locations)
    #sampling_range = np.unique(np.logspace(0, np.log10(neighborhood_size), n_datapoints).astype(int))
    sampling_range = range(1, neighborhood_size, round(neighborhood_size / n_datapoints))
    
    norm_factors = labels.value_counts() / len(labels) * len(labels.value_counts())

    #neighborhood_df = pd.DataFrame(columns=sampling_range, index=adata.obs['ObjectNumber'])

    def get_neighborhood_series(index, celltype):
        neighbors = pd.Series(dist_matrix[index], index=labels).sort_values()
        if normalized:
            return [1 - (neighbors[:i].index.value_counts()[celltype] / i / norm_factors[celltype] ) for i in sampling_range]
        return [1 - (neighbors[:i].index.value_counts()[celltype] / i)  for i in sampling_range]

    neighborhood_df = pd.DataFrame(Parallel(n_jobs=n_jobs)(delayed(get_neighborhood_series)(index, celltype) for index, celltype in tqdm(enumerate(labels))),
                                              columns=sampling_range, index=labels)
    
    #print(neighborhood_df)
    summary = pd.concat([neighborhood_df.mean(axis=0), neighborhood_df.std(axis=0)], axis=1)
    summary.columns = ["mean", "sd"]
    summary['neighborhood'] = np.linspace(0, 1, len(summary))

    if color == 'auto':
        line = ax.plot(summary.neighborhood, summary['mean'], label=label, linestyle=linestyle)
    else:
        line = ax.plot(summary.neighborhood, summary['mean'], label=label, linestyle=linestyle, color=color)

    ax.set_xlabel('Relative neighborhood size')
    ax.set_ylabel('Cluster intermixing')

    return line

In [ ]:
def general_intermixing_metric_sampled(locations, labels, ax, label, sample_frac = 1, linestyle = 'solid', color = 'auto', n_datapoints = 100, normalized = True, n_jobs = 6):
    
    sample = pd.DataFrame({'cell': range(len(labels)), 'label': labels}).groupby('label', group_keys=False).apply(lambda x: x.sample(frac=sample_frac, random_state=1))
 
    sample_locations = locations[sample.cell]
    sample_labels = labels[sample.cell]
    
    dist_matrix = distance_matrix(sample_locations, locations)
    neighborhood_size = len(locations)
    #sampling_range = np.unique(np.logspace(0, np.log10(neighborhood_size), n_datapoints).astype(int))
    sampling_range = range(1, neighborhood_size, round(neighborhood_size / n_datapoints))
    
    norm_factors = labels.value_counts() / len(labels) * len(labels.value_counts())

    #neighborhood_df = pd.DataFrame(columns=sampling_range, index=adata.obs['ObjectNumber'])

    def get_neighborhood_series(index, celltype):
        neighbors = pd.Series(dist_matrix[index], index=labels).sort_values()
        if normalized:
            return [1 - (neighbors[:i].index.value_counts()[celltype] / i / norm_factors[celltype] ) for i in sampling_range]
        return [1 - (neighbors[:i].index.value_counts()[celltype] / i)  for i in sampling_range]

    neighborhood_df = pd.DataFrame(Parallel(n_jobs=n_jobs)(delayed(get_neighborhood_series)(index, celltype) for index, celltype in tqdm(enumerate(sample_labels))),
                                              columns=sampling_range, index=sample_labels)
    
    #print(neighborhood_df)
    summary = pd.concat([neighborhood_df.mean(axis=0), neighborhood_df.std(axis=0)], axis=1)
    summary.columns = ["mean", "sd"]
    summary['neighborhood'] = np.linspace(0, 1, len(summary))

    if color == 'auto':
        line = ax.plot(summary.neighborhood, summary['mean'], label=label, linestyle=linestyle)
    else:
        line = ax.plot(summary.neighborhood, summary['mean'], label=label, linestyle=linestyle, color=color)

    ax.set_xlabel('Relative neighborhood size')
    ax.set_ylabel('Cluster intermixing')

    return line

### 2. The datasets

To examine the behavior of the metric with different degrees of intermixing, population sizes etc., random datasets are produced. With varying parameters for distance between populations, size of the whole set, fractional proportions and spread of the two populations, this function generates two sets of normaly distributed points in 2D space. 

In [ ]:
def artificial_dataset(label, distance = 5, size = 1000, frac_1 = 0.5, spread = (1, 1)):
    
    np.random.seed(1)
    xy_dist = np.sqrt(distance**2 / 2)

    pop1_size = round(frac_1 * size)
    pop2_size = size - pop1_size

    pop1 = np.reshape(np.random.normal(0, spread[0], pop1_size*2), (pop1_size, 2))
    pop2 = np.reshape(np.random.normal(xy_dist, spread[1], pop2_size*2), (pop2_size, 2))

    fig, ax3 = plt.subplots()
    ax3.scatter(pop1[:, 0], pop1[:, 1], label='population1')
    ax3.scatter(pop2[:, 0], pop2[:, 1], label='population2')
    ax3.set_title(label)
    
    locations = np.concatenate((pop1, pop2))
    labels = pd.Series(np.concatenate((['a'] * pop1_size, ['b'] * pop2_size)))
    return (locations, labels, label)

def artificial_line(label, distance = 5, size = 100):
    
    coord = np.linspace(0, distance, size)
    
    fig, ax3 = plt.subplots()
    ax3.scatter(coord[[i*2 for i in range(0, int(size/2))]], coord[[i*2 for i in range(0, int(size/2))]], label='population1')
    ax3.scatter(coord[[i*2+1 for i in range(0, int(size/2))]], coord[[i*2+1 for i in range(0, int(size/2))]], label='population2')
    ax3.set_title(label)

    locations = np.transpose(np.vstack((coord, coord)))
    labels = pd.Series(['a', 'b'] * int(size/2))
    return (locations, labels, label)



### 3. Examination

As a fist glance, two partly overlapping populations are generated at different proportions (between 50:50 and 90:10). Their intermixing metrics are summarised as solid lines that rise from 0 at $n_{rel}=0$ for increasing neighborhood sizes to different end points at $n_{rel}=1$ (0.5, ~0.4, ~0.15). This shows that the metric is dependant on the relative population sizes. To circumvent this, the metric can be normalized by dividing the individual intermixing metrics $i_{a,n}$ by their relative population sizes. That way, the same datasets produce intermixing curves that all range between $[0;0.5]$. This normalization would also concur with the visual impression, that the two populations appear more confined for 50:50 and 70:30 and less for 90:10.

In [ ]:
from locale import normalize

plt.style.use('tableau-colorblind10')


fig, ax = plt.subplots()

locations, labels, label = artificial_dataset('dist-3_spread-1_50:50', distance=3, frac_1=0.5)
general_intermixing_metric(locations, labels, ax, label, normalized = False)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

locations, labels, label = artificial_dataset('dist-3_spread-1_70:30', distance=3, frac_1=0.7)
general_intermixing_metric(locations, labels, ax, label, normalized = False)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

locations, labels, label = artificial_dataset('dist-3_spread-1_90:10', distance=3, frac_1=0.9)
general_intermixing_metric(locations, labels, ax, label, normalized = False)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

ax.legend()
fig.tight_layout()

Next, populations with the same distribution parameters but different degrees of overlap are compared. While two populations with no difference in mean reach a high intermixing score for very small neighborhoods, the metric remains around zero almost half way for a mean distance of 5 standard deviations.

In [ ]:

fig, ax = plt.subplots()

locations, labels, label = artificial_dataset('dist-5_spread-1_50:50', distance=5)
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('dist-4_spread-1_50:50', distance=4)
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('dist-3_spread-1_50:50', distance=3)
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('dist-2_spread-1_50:50', distance=2)
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('dist-1_spread-1_50:50', distance=1)
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('dist-0_spread-1_50:50', distance=0)
general_intermixing_metric(locations, labels, ax, label)

ax.legend()
fig.tight_layout()

Two extreme scenarios are shown here: If the distance between populations is greater then the longest distances within the populations, the intermixing curve remains zero for half of the dataset size. Conversely, if data points are positioned in alternating order (in this case on a line), the closest neighbors of almost all points are from the other population, thus the intermixing metric goes up immediately and reaches values higher than 0.5.

In [ ]:
fig, ax = plt.subplots()

locations, labels, label = artificial_dataset('no_spread', distance=1, spread=(0, 0))
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_line('line', distance=5, size=100)
general_intermixing_metric(locations, labels, ax, label)

ax.legend()
fig.tight_layout()

In order to further investigate the comparability of different spatial configurations, an unbalanced set is analyzed at different levels of cluster spread and distance. Although both parameter have an influence on the intermixing metric, the data spread seems to be much more prominent than the localization of the clusters relative to each other. 

In [ ]:
fig, ax = plt.subplots()


locations, labels, label = artificial_dataset('90_10', distance=0, frac_1=0.9)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

locations, labels, label = artificial_dataset('90_10_distance_1', distance=1, frac_1=0.9)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

locations, labels, label = artificial_dataset('90_10_distance_2', distance=2, frac_1=0.9)
general_intermixing_metric(locations, labels, ax, label, linestyle='dashed')

locations, labels, label = artificial_dataset('90_10_dense', distance=0, frac_1=0.9, spread=(1, 0.1))
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('90_10_dense_distance_1', distance=1, frac_1=0.9, spread=(1, 0.1))
general_intermixing_metric(locations, labels, ax, label)

locations, labels, label = artificial_dataset('90_10_dense_distance_2', distance=2, frac_1=0.9, spread=(1, 0.1))
general_intermixing_metric(locations, labels, ax, label)

ax.legend()
fig.tight_layout()

With the present implementation, the performance of the metric's calculation depends strongly on the number of datapoints in the set (the neighborhood of every one is examined). In order to speed up the process, one can take a random sample, stratified by population and calculate the intermixing metric for this sample. In the following, the full set metric is compared with sampled variants (30%, 10%, 3% sampling fraction). As visible in the plot, the information value of the metric is retained even for lower sampling fractions.

In [ ]:
plt.style.use('tableau-colorblind10')

fig, ax = plt.subplots()

locations, labels, label = artificial_dataset('dist-3_spread-1_50:50', distance=3, frac_1=0.5)
general_intermixing_metric(locations, labels, ax, label, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.3', sample_frac=0.3, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.1', sample_frac=0.1, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.03', sample_frac=0.03, normalized = False)

locations, labels, label = artificial_dataset('dist-2_spread-1_50:50', distance=2, frac_1=0.5)
general_intermixing_metric(locations, labels, ax, label, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.3', sample_frac=0.3, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.1', sample_frac=0.1, normalized = False)
general_intermixing_metric_sampled(locations, labels, ax, label+'_frac0.03', sample_frac=0.03, normalized = False)

ax.legend()